#FLASK SERVER

In [ ]:
# For public server
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=41f5d141af298f8ae5277aa3f2970b4ee9eb9a8dd0ed3c49f252b6afec8a90c7
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
from flask import Flask, request, Response, jsonify
from pyngrok import ngrok
# from multiprocessing import Process
import threading
import base64
import os
import shutil

In [ ]:
# List images as json
def list_images(folder, image_type):
    image_files = os.listdir(folder)
    images = []
    for file_name in image_files:
        with open(os.path.join(folder, file_name), 'rb') as f:
            image_bytes = f.read()
            images.append({
                'name': file_name,
                'type': image_type,
                'data': base64.b64encode(image_bytes).decode('utf-8')
            })
    return images

# Recreate folder
def recreate(folder):
    # Check if the folder exists
    if os.path.exists(folder):
        # Remove the folder and its contents
        shutil.rmtree(folder)
    # Create a single directory
    os.mkdir(folder)


In [ ]:
input_path = '/content/input/input.mp4'
output_path = '/content/output/output.mp4'
intermid_output = '/content/inter_output2.mp4'

input_criminal_folder = '/content/drive/MyDrive/TheProject/criminal4'
image_folder = '/content/image'
output_face_folder = '/content/face'
fall_image_folder = '/content/fall'
crash_folder = '/content/crash'

# 'begin', 'finished', 'processing'
status = 'begin'

frame_count = 1
frame_number = -1
my_model = ''
id = ''

def process():
    global input_criminal_folder
    global image_folder
    global output_face_folder
    global fall_image_folder
    global mask_image_folder
    global crash_folder
    global my_model
    global status

    global frame_number
    global frame_count

    # Clear folders
    recreate(image_folder)
    recreate(output_face_folder)
    recreate(fall_image_folder)
    recreate(crash_folder)

    start=time.time()
    try:
        status = 'processing'
        if my_model == 'fight':
            list_criminal_images = os.listdir(input_criminal_folder)

            fightDetection2(input_path,output_path,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=False)

        # Fight, Face
        elif my_model == 'fight_face':
            list_criminal_images = os.listdir(input_criminal_folder)
            fightDetection2(input_path,output_path,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=True)

        # Fastest with max models number
        # Fight, Face, Fall, Crash
        elif my_model == 'fight_fall_crash':
            list_criminal_images = os.listdir(input_criminal_folder)
            fightDetection2(input_path,intermid_output,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=False)
            model_call(fall_crash,intermid_output,output_path,fall_image_folder,crash_folder,mask_model=False)

        # Fall, crash
        elif my_model == 'fall_crash':
            model_call(fall_crash,input_path,output_path,fall_image_folder,crash_folder)

        # Fall, crash, mask
        elif my_model == 'fall_crash_mask':
            model_call(fall_crash,input_path,output_path,fall_image_folder,crash_folder,mask_model=True)

        # Fight, Face, Fall
        elif my_model == 'indoor':
            list_criminal_images = os.listdir(input_criminal_folder)

            fightDetection2(input_path,intermid_output,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=True)
            model_call(human_fall,intermid_output,output_path,fall_image_folder,crash_folder,mask_model=False)

        # Fight, Face, Fall, Mask
        elif my_model == 'indoor_mask':
            list_criminal_images = os.listdir(input_criminal_folder)

            fightDetection2(input_path,intermid_output,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=True)
            model_call(human_fall,intermid_output,output_path,fall_image_folder,crash_folder,mask_model=True)

        # Fight, Face, Fall, Mask
        elif my_model == 'fight_fall_mask':
            list_criminal_images = os.listdir(input_criminal_folder)

            fightDetection2(input_path,intermid_output,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=False)
            model_call(human_fall,intermid_output,output_path,fall_image_folder,crash_folder,mask_model=True)

        # Fight, Face, Fall, Crash
        elif my_model == 'general':
            list_criminal_images = os.listdir(input_criminal_folder)

            fightDetection2(input_path,intermid_output,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=True)
            model_call(fall_crash,intermid_output,output_path,fall_image_folder,crash_folder,mask_model=False)

        # Fight, Face, Fall, Crash, Mask
        elif my_model == 'general_mask':
            list_criminal_images = os.listdir(input_criminal_folder)

            fightDetection2(input_path,intermid_output,image_folder,input_criminal_folder,list_criminal_images,output_face_folder,face_model=True)
            model_call(fall_crash,intermid_output,output_path,fall_image_folder,crash_folder,mask_model=True)

        status = 'finished'
    except Exception as e:
        frame_number = -1
        frame_count = 1
        print(e)
        return e
    elapsed=time.time()-start

    # else:
    #     return "Must enter model in the header...", 400
    print("-------------------------------processing done successfully-------------------------------")
    print("Take:",f'{elapsed:.2f}'," Second")
    print("Take:",f'{elapsed/60:.2f}'," min")

In [ ]:

app = Flask(__name__)

def is_processing():
    global frame_number
    global frame_count

    return (not (frame_number < 0) and frame_number*100/frame_count < 100 and status=='processing')


@app.route('/')
def test():
    global frame_number
    global frame_count
    percent = frame_number * 100 / frame_count

    # this ="<h1>The app is working frame_number = {}, frame_count = {}, percent = {} </h1>".format(frame_number,frame_count, frame_number*100/frame_count)
    return "<h1>The app is working. frame_number = {}, frame_count = {}, percent = {}, model = {}</h1>".format(frame_number, frame_count, f'{percent:.2f}', my_model)

@app.route('/input', methods=['POST'])
def video():
    global frame_number
    global frame_count
    global status

    response_data = {}
    video_bytes = 0
    if is_processing():
        return 'wait for the last process to finish',400

    content_type = request.headers.get('Content-Type')
    print(content_type)
    if content_type.startswith('multipart/form-data'):
        # Receive the byte array from the HTTP request
        files = request.files
        video_bytes = files['video'].read()
        print("Received Video in files")

    elif content_type == 'application/json':
        json_data = request.json
        video_bytes = base64.b64decode(json_data['video'])
        print("Received Video in json")

    elif content_type == 'application/octet-stream' or content_type =='video/mp4':
        video_bytes = request.data
        print("Received Video in data")

    else:
        print("Error Not received the video...")
        return "Error Not received the video...", 400

    # Pathes
    global input_path
    global output_path

    if video_bytes:
        recreate('/content/input')
        # convert the video back to file
        with open(input_path, 'wb') as f:
            f.write(video_bytes)
    else:
        print("Error Not received the video...")
        return "Error Not received the video...", 400

    global id
    global my_model

    my_model = request.headers.get('X-model')
    id = request.headers.get('X-id')
    if not my_model:
        print("Error Not received the model type...")
        return "Error Not received the model type...", 400
    recreate('/content/output')

    status = 'begin'
    model_func(is_server_call=True)

    print('Video was received successfully')
    return 'Video was received successfully and have been processed...'

@app.route('/model', methods=['POST'])
def model_func(is_server_call=False):
    global input_path
    global my_model

    global frame_number
    global frame_count
    if is_server_call:
        print('model type choosed successfully to process The last video...')

    else:
        if is_processing():
            return 'Please wait for the last process to finish',400

        if not os.path.exists(input_path):
            return 'Please send input video first to /input route!',400

        my_model = request.headers.get('X-model')
        if not my_model:
            return "Error Not received the model type...", 400
        print('model type choosed successfully to process the last video...')

    threading.Thread(target=process).start()

    return 'model type choosed successfully to process the last video...'


@app.route('/response', methods=['POST'])
def response():
    print(status)
    global input_path
    global frame_number
    global frame_count
    global my_model

    global image_folder
    global output_face_folder
    global fall_image_folder
    global mask_image_folder

    # Return The Response  TO Continue<--
    response_data= {}
    data = {}
    response_data['status'] = 'failed'
    response_data['percent'] = 0
    response_data['trigger'] = False
    response_data['data'] = None
    response_data['msg'] = 'None'
    response_data['model'] = my_model
    response_data['id'] = id

    if frame_number < 0:
        response_data['status'] = 'failed'
        response_data['percent'] = 0
        response_data['trigger'] = False
        response_data['data'] = None
        response_data['msg'] = 'Please send input video and model first to /input route!'
        response = jsonify(response_data)
        return response

    percent = frame_number*100/frame_count
    response_data['percent'] = percent

    if percent < 100 and status == 'processing':
        response_data['status'] = 'processing'
        data['video'] = None
        data['images'] = None

    elif status == 'finished':
        response_data['status'] = 'finished'
        response_data['trigger'] = True

        # Adding video
        # Convert the video back to a byte array
        with open(output_path, 'rb') as f:
            processed_video_bytes = f.read()

        # processed_video_bytes variable as a string before passing it to jsonify()
        data['video'] = base64.b64encode(processed_video_bytes).decode('utf-8')

        # Adding images folder to response
        data['images'] = list_images(image_folder,'fight')
        data['images'] += list_images(output_face_folder,'face')
        data['images'] += list_images(fall_image_folder,'fall')
        data['images'] += list_images(crash_folder,'crash')

    response_data['msg'] = 'response data sent successfully'
    response_data['data'] = data
    response = jsonify(response_data)
    return response


In [ ]:
port_no = 8000

ngrok.set_auth_token("2ROjGpubBNIJvRPZ5FKz5cUGg22_6WWHGJe2LDxuzPQ8wRXAc")

# Run the App
public_url =  ngrok.connect(port_no).public_url
if __name__ == "__main__":
    print(f"To acces the Gloable link please click {public_url}")
    app.run(port=port_no)

To acces the Gloable link please click https://f4ec-34-143-197-241.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
